# <font color='violet'> Deeper EDA on Review Text

On data that was cleaned and lemmatized here: https://github.com/fractaldatalearning/psychedelic_efficacy/blob/main/notebooks/4-kl-studies-nlp.ipynb


    
    Resources with tips for effective EDA visualization with NLP:

https://medium.com/plotly/nlp-visualisations-for-clear-immediate-insights-into-text-data-and-outputs-9ebfab168d5b
    
https://www.numpyninja.com/post/nlp-text-data-visualization
    
https://www.kaggle.com/code/sainathkrothapalli/nlp-visualisation-guide
    
https://medium.com/acing-ai/visualizations-in-natural-language-processing-2ca60dd34ce
    
https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
    
https://towardsdatascience.com/getting-started-with-text-nlp-visualization-9dcb54bc91dd
    
https://www.kaggle.com/code/mitramir5/nlp-visualization-eda-glove
    
https://medium.com/analytics-vidhya/how-to-begin-performing-eda-on-nlp-ffdef92bedf6
    
https://inside-machinelearning.com/en/eda-nlp/
    
https://towardsdatascience.com/fundamental-eda-techniques-for-nlp-f81a93696a75
    
https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools
    
https://www.kdnuggets.com/2019/05/complete-exploratory-data-analysis-visualization-text-data.html
    


In [9]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('../data/interim/studies_lemmatized.csv').drop(columns=['Unnamed: 0'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31559 entries, 0 to 31558
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rating         31559 non-null  float64
 1   condition      31559 non-null  object 
 2   review         31559 non-null  object 
 3   date           31451 non-null  object 
 4   drug0          31559 non-null  object 
 5   drug1          18992 non-null  object 
 6   drug2          32 non-null     object 
 7   drug3          23 non-null     object 
 8   drug4          12 non-null     object 
 9   drug5          11 non-null     object 
 10  drug6          7 non-null      object 
 11  drug7          5 non-null      object 
 12  drug8          3 non-null      object 
 13  drug9          2 non-null      object 
 14  drug10         2 non-null      object 
 15  drug11         2 non-null      object 
 16  drug12         2 non-null      object 
 17  drug13         2 non-null      object 
 18  drug14

How do the lengths compare for the review strings with just punctuation removed vs. the lemmatized strings?

In [10]:
review_lengths = [len(df.loc[row,'review']) for row in range(len(df))]
review_mean_length = np.mean(review_lengths)
review_mean_length

457.6392471244336

In [11]:
lemm_length = [len(df.loc[row,'no_stops_lemm']) for row in range(len(df))]
lemm_mean_length = np.mean(lemm_length)
lemm_mean_length

256.8190690452803